In [12]:
import pandas as pd
import datetime
from sqlalchemy import create_engine
from sqlalchemy import inspect
from login import pw

In [13]:
#import data and ignore the first 11 rows except Row 2
inflation_df = pd.read_csv('Resources/inflation.csv',encoding='cp1252', skiprows=[0,2,3,4,5,6,7,8,9,10])

In [14]:
inflation_df.head(20)

,Title,Consumer price index,Year-ended inflation,Year-ended inflation – excluding interest and tax changes,Year-ended inflation – excluding volatile items,Year-ended tradables inflation,Year-ended tradables inflation – excluding volatile items and tobacco,Year-ended non-tradables inflation,Year-ended non-tradable inflation – excluding interest charges and deposit & loan facilities,Year-ended weighted median inflation,...,Quarterly inflation,Quarterly inflation – excluding interest and tax changes,Quarterly inflation – excluding volatile items,Quarterly tradables inflation,Quarterly tradables inflation – excluding volatile items and tobacco,Quarterly non-tradables inflation,Quarterly non-tradables inflation – excluding deposit and loan facilities,Quarterly weighted median inflation,Quarterly trimmed mean inflation,Unnamed: 21
0,Jun-1922,2.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sep-1922,2.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dec-1922,2.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mar-1923,2.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Jun-1923,2.8,2.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Sep-1923,2.9,3.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Dec-1923,2.9,4.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Mar-1924,2.8,1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Jun-1924,2.8,-1.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Sep-1924,2.8,-3.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
inflation_new_df = inflation_df[['Title','Consumer price index','Year-ended inflation','Quarterly tradables inflation','Quarterly non-tradables inflation','Quarterly trimmed mean inflation']].copy()

In [16]:
inflation_new_df.head()

,Title,Consumer price index,Year-ended inflation,Quarterly tradables inflation,Quarterly non-tradables inflation,Quarterly trimmed mean inflation
0,Jun-1922,2.8,NaN,NaN,NaN,NaN
1,Sep-1922,2.8,NaN,NaN,NaN,NaN
2,Dec-1922,2.7,NaN,NaN,NaN,NaN
3,Mar-1923,2.7,NaN,NaN,NaN,NaN
4,Jun-1923,2.8,2.3,NaN,NaN,NaN


In [17]:

inflation_new_df = inflation_new_df.rename(columns={"Title":"date","Consumer price index": "consumer_price_index","Year-ended inflation":"year_end_inflation",
                                                          "Quarterly tradables inflation": "quar_trade_inflation",
                                                          "Quarterly trimmed mean inflation": "quar_mean_inflation"})


In [18]:
inflation_new_df.dropna(inplace=True)

In [19]:
inflation_new_df.head()

,date,consumer_price_index,year_end_inflation,quar_trade_inflation,Quarterly non-tradables inflation,quar_mean_inflation
240,Jun-1982,31.5,10.9,1.6,3.4,2.5
241,Sep-1982,32.6,12.4,3.3,3.7,3.0
242,Dec-1982,33.6,11.3,2.0,3.4,2.6
243,Mar-1983,34.3,11.4,2.5,2.2,2.2
244,Jun-1983,35.0,11.1,2.9,1.6,1.8


In [20]:
inflation_new_df['date']= pd.to_datetime(inflation_new_df['date'])

In [33]:
inflation_new_df['quarter']= pd.PeriodIndex(inflation_new_df.date, freq ='Q')
inflation_new_df['quarter']=inflation_new_df['quarter'].astype(str)
inflation_new_df

,date,consumer_price_index,year_end_inflation,quar_trade_inflation,Quarterly non-tradables inflation,quar_mean_inflation,quarter
240,1982-06-01,31.5,10.9,1.6,3.4,2.5,1982Q2
241,1982-09-01,32.6,12.4,3.3,3.7,3.0,1982Q3
242,1982-12-01,33.6,11.3,2.0,3.4,2.6,1982Q4
243,1983-03-01,34.3,11.4,2.5,2.2,2.2,1983Q1
244,1983-06-01,35.0,11.1,2.9,1.6,1.8,1983Q2
...,...,...,...,...,...,...,...
396,2021-06-01,118.8,3.8,1.2,0.5,0.4,2021Q2
397,2021-09-01,119.7,3.0,0.8,1.0,0.9,2021Q3
398,2021-12-01,121.3,3.5,1.9,1.1,1.0,2021Q4
399,2022-03-01,123.9,5.1,2.8,1.6,1.5,2022Q1


In [34]:
inflation_new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161 entries, 240 to 400
Data columns (total 7 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   date                               161 non-null    datetime64[ns]
 1   consumer_price_index               161 non-null    float64       
 2   year_end_inflation                 161 non-null    float64       
 3   quar_trade_inflation               161 non-null    float64       
 4   Quarterly non-tradables inflation  161 non-null    float64       
 5   quar_mean_inflation                161 non-null    float64       
 6   quarter                            161 non-null    object        
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 10.1+ KB


In [35]:
protocol = 'postgresql'
username = 'postgres'
host = 'localhost'
port = 5432
database_name = 'project2_db'
rds_connection_string = f'{protocol}://{username}:{pw}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)

In [36]:
insp.get_table_names()

['inflation']

In [37]:
inflation_new_df.to_sql(name='inflation',con=engine, if_exists='replace',index=False)

161

In [27]:
pd.read_sql_query('select * from inflation', con=engine).head()

,date,consumer_price_index,year_end_inflation,quar_trade_inflation,Quarterly non-tradables inflation,quar_mean_inflation,quarter
0,1982-06-01,31.5,10.9,1.6,3.4,2.5,1982Q2
1,1982-09-01,32.6,12.4,3.3,3.7,3.0,1982Q3
2,1982-12-01,33.6,11.3,2.0,3.4,2.6,1982Q4
3,1983-03-01,34.3,11.4,2.5,2.2,2.2,1983Q1
4,1983-06-01,35.0,11.1,2.9,1.6,1.8,1983Q2
